In [24]:
#PERMANOVA for the statistical analysis of Extracted Audio Features using the Mean and Median data for each year
#Justin T. Niestroj
#University of Hamburg
#2025

import pandas as pd #for data analysis and visualization
import numpy as np #for calculations and mathmatical functions
from scipy.spatial.distance import pdist, squareform #for statistical calculations and eucledian distance calculations
import matplotlib.pyplot as plt #for data analysis and visualization
import io #for implementation of plots in html
import base64 #for implementation of plots in html

#calculation of PERMANOVA

def permanova_manual(data: pd.DataFrame, grouping: pd.Series, permutations: int = 999):
    distance_matrix = squareform(pdist(data.values, metric='euclidean')) #calculates the euclidean distance between each data point

    #calculated the total sum of squares for each group
    n = len(data)
    grand_mean = np.mean(distance_matrix[np.triu_indices(n, k=1)])
    SST = np.sum((distance_matrix[np.triu_indices(n, k=1)] - grand_mean)**2)

    #within each group, calculates the distance within the group
    groups = grouping.values
    unique_groups = np.unique(groups)
    SSw = 0
    for group in unique_groups:
        idx = np.where(groups == group)[0]
        if len(idx) > 1:
            group_dist = distance_matrix[np.ix_(idx, idx)]
            group_mean = np.mean(group_dist[np.triu_indices(len(idx), k=1)])
            SSw += np.sum((group_dist[np.triu_indices(len(idx), k=1)] - group_mean)**2) #within group variation

    #defines the difference if not within then between groups and calculates the F-Statistic
    SSb = SST - SSw
    df_between = len(unique_groups) - 1
    df_within = n - len(unique_groups)
    F_stat = (SSb / df_between) / (SSw / df_within)

    #permutation training
    perm_stats = []
    for _ in range(permutations):
        shuffled = np.random.permutation(groups)
        SSw_perm = 0
        for group in unique_groups:
            idx = np.where(shuffled == group)[0]
            if len(idx) > 1:
                group_dist = distance_matrix[np.ix_(idx, idx)]
                group_mean = np.mean(group_dist[np.triu_indices(len(idx), k=1)])
                SSw_perm += np.sum((group_dist[np.triu_indices(len(idx), k=1)] - group_mean)**2)
        SSb_perm = SST - SSw_perm
        F_perm = (SSb_perm / df_between) / (SSw_perm / df_within)
        perm_stats.append(F_perm)

    #calculates the p-value (statistical significance) based on the permutation
    p_value = np.mean([f >= F_stat for f in perm_stats])

    #returns the results
    return {
        'F': round(F_stat, 4),
        'p-value': round(p_value, 4),
        'df_between': df_between,
        'df_within': df_within,
        'permutations': permutations
    }

#loads and prepares data from the extracted csv files and labels the data by country, decade and year
df_us = pd.read_csv(r"C:\Users\Justin\Documents\MA\Code\SOM year mean csv\yearly_feature_means_US.csv")
df_us['country'] = 'US'
df_ger = pd.read_csv(r"C:\Users\Justin\Documents\MA\Code\SOM year mean csv\yearly_feature_means_GER.csv")
df_ger['country'] = 'DE'
df = pd.concat([df_us, df_ger], ignore_index=True)
df['decade'] = (df['Year'] // 10) * 10

#defines the features being used; if features are deleted, the PERMANOVA will be calculated with the remaining features
feature_cols = [
    'CrestFactorLow', 
    'CrestFactorMid', 
    'CrestFactorHigh',
    'ChannelCorrelationLow', 
    'ChannelCorrelationMid', 
    'ChannelCorrelationHigh',
    'RMSLow',
    'RMSMid',
    'RMSHigh',
    'PhaseSpaceLow',
    'PhaseSpaceMid',
    'PhaseSpaceHigh', 
    'PeakMeterLow'
    'PeakMeterMid', 
    'PeakMeterHigh', 
]

#prepares group labels
features = df[feature_cols]
group_country = df['country']
group_decade = df['decade'].astype(str)
group_interaction = df['country'] + "_" + group_decade

#runs PERMANOVA
res_country = permanova_manual(features, group_country)
res_decade = permanova_manual(features, group_decade)
res_interaction = permanova_manual(features, group_interaction)

#results for differences per decade
results_per_decade = []
for decade in sorted(df['decade'].unique()):
    df_decade = df[df['decade'] == decade]
    if df_decade['country'].nunique() >= 2:
        result = permanova_manual(df_decade[feature_cols], df_decade['country'])
        results_per_decade.append({'Decade': str(decade), 'F-statistic': result['F'], 'p-value': result['p-value']})

#results for differences in rolling 5-year window
rolling_results = []
for start in range(df['Year'].min(), df['Year'].max() - 4):
    end = start + 4
    df_block = df[(df['Year'] >= start) & (df['Year'] <= end)]
    if df_block['country'].nunique() >= 2:
        result = permanova_manual(df_block[feature_cols], df_block['country'])
        rolling_results.append({'5-Year Window': f"{start}-{end}", 'F-statistic': result['F'], 'p-value': result['p-value']})

#transforms results to data frames
df_summary = pd.DataFrame([
    {'Test': 'Country', 'F-statistic': res_country['F'], 'p-value': res_country['p-value']},
    {'Test': 'Decade', 'F-statistic': res_decade['F'], 'p-value': res_decade['p-value']},
    {'Test': 'Country × Decade', 'F-statistic': res_interaction['F'], 'p-value': res_interaction['p-value']}
])

df_decade_results = pd.DataFrame(results_per_decade)
df_roll = pd.DataFrame(rolling_results)
df_roll['mid_year'] = df_roll['5-Year Window'].apply(lambda x: int(x.split('-')[0]) + 2)

#converts results to base64 to write in hmtl files
def plot_to_base64():
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)
    return base64.b64encode(buf.read()).decode('utf-8')

#creating Plot 1: Overall Summery over the entire time frame
plt.figure(figsize=(8, 5))
bars = plt.bar(df_summary['Test'], df_summary['F-statistic'], color=['#6699cc', '#66cc99', '#cc6666'])
plt.title("PERMANOVA Test Statistics")
plt.ylabel("F-statistic")
for bar, p in zip(bars, df_summary['p-value']):
    height = bar.get_height()
    label = f"p = {p:.4f}" if p >= 0.0001 else "p < 0.0001"
    plt.text(bar.get_x() + bar.get_width()/2, height + 1, label, ha='center', va='bottom')
img_summary = plot_to_base64()

#creating Plot 2: Country Differences per Decade
plt.figure(figsize=(8, 5))
bars = plt.bar(df_decade_results['Decade'], df_decade_results['F-statistic'], color='skyblue')
plt.title("PERMANOVA: Country Difference per Decade")
plt.xlabel("Decade")
plt.ylabel("F-statistic")
for i, row in df_decade_results.iterrows():
    label = f"p = {row['p-value']:.4f}" if row['p-value'] >= 0.0001 else "p < 0.0001"
    plt.text(i, row['F-statistic'] + 1, label, ha='center', fontsize=9)
img_decade = plot_to_base64()

#creating Plot 3: Country Differences in 5-year rolling window
plt.figure(figsize=(10, 5))
plt.plot(df_roll['mid_year'], df_roll['F-statistic'], marker='o', color='steelblue')
plt.title("PERMANOVA F-statistic by Country (5-Year Rolling Windows)")
plt.xlabel("Mid-Year of 5-Year Block")
plt.ylabel("F-statistic")
plt.grid(True)
for _, row in df_roll.iterrows():
    if row['p-value'] < 0.05:
        plt.text(row['mid_year'], row['F-statistic'] + 1, "*", ha='center', fontsize=14, color='red')
img_rolling = plot_to_base64()

#write html file with results as data tables and graphs
with open("permanova_report_embedded.html", "w", encoding="utf-8") as f:
    f.write("<html><head><title>PERMANOVA Report</title></head><body>")
    f.write("<h1>🎯 PERMANOVA Overall Summary</h1>")
    f.write(df_summary.to_html(index=False, float_format="%.4f"))
    f.write(f'<img src="data:image/png;base64,{img_summary}"><hr>')

    f.write("<h2>🕰️ PERMANOVA by Decade</h2>")
    f.write(df_decade_results.to_html(index=False, float_format="%.4f"))
    f.write(f'<img src="data:image/png;base64,{img_decade}"><hr>')

    f.write("<h2>📉 PERMANOVA 5-Year Rolling Window</h2>")
    f.write(df_roll[['5-Year Window', 'F-statistic', 'p-value']].to_html(index=False, float_format='%.4f'))
    f.write(f'<img src="data:image/png;base64,{img_rolling}"><hr>')

    f.write("</body></html>")

print("\n✅ Report with inline plots saved as: permanova_report_embedded.html")



✅ Report with inline plots saved as: permanova_report_embedded.html
